In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from time import sleep
import json
import logging
import datetime
from selenium.webdriver.chrome.options import Options

import subprocess
import gzip
import os
import json
from time import sleep

from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import json
import datetime
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
from time import sleep
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import random


In [3]:
def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data



import re

def get_competition(text):
    # 假设你已经用之前的方法获得了这个字符串,字符串样例示例：
    #text = "（排名129/364）"
    # 使用正则表达式匹配文本中的数字
    matches = re.findall(r'\d+', text)

    # 假设文本格式固定，排名和总人数都是数字，并且总人数在第二个位置
    if matches:
        total_number = matches[1]  # 获取总人数
        
    return total_number



In [4]:

options = ChromeOptions()
""" #使用代理服务器，便于用Mitmproxy监听
options.add_argument("--proxy-server=http://localhost:8080") """
# 忽略SSL错误
options.add_argument("--ignore-ssl-errors=yes")
options.add_argument("--ignore-certificate-errors")
#禁用Blink引擎的某些特性，未指定
options.add_argument("--disable-blink-features")
#隐藏Selenium自动化控制的痕迹
options.add_argument("--disable-blink-features=AutomationControlled")

############
#无界面模式
#options.add_argument("--headless")
############

chromedriver_path = r"C:\chromedriver-win64\chromedriver.exe"


""" try:
    times = int(input("输入需要爬取的数量，没有输入则爬取所有岗位"))
except ValueError:
    times = float('inf')  # 设置为无限大，表示爬取所有职位
 """
times = float('inf')

# 创建Service对象，并传入ChromeDriver的路径
service = Service(executable_path=chromedriver_path)
# 使用Service对象初始化Chrome WebDriver
driver = webdriver.Chrome(service=service,options=options)

wait = WebDriverWait(driver, 60)


In [ ]:
driver()

In [25]:
import sys

def get_information_from_elements(driver,job_listings):
    elements = driver.find_elements(By.CSS_SELECTOR, ".e")
    ele_nums=0
        # 遍历每个元素，提取并保存感兴趣的信息
    for element in elements:
        
        try:
            job_title = element.find_element(By.CSS_SELECTOR, "a.zhn").text
            salary = element.find_element(By.CSS_SELECTOR, "span.xz").text
            company = element.find_element(By.CSS_SELECTOR, "a.gs").text
            location = element.find_element(By.CSS_SELECTOR, "span.dq").text
            application_time = element.find_element(By.CSS_SELECTOR, ".li.l2 .rq span").text
            resume_status = element.find_element(By.CSS_SELECTOR, ".li.l4 .view span").text
            
            # 创建包含所有信息的字典
            job_info = {
                "Job Title": job_title,
                "Salary": salary,
                "Company": company,
                "Location": location,
                "Application Time": application_time,
                "Resume Status": resume_status
            }
            
            # 将字典添加到列表中
            job_listings.append(job_info)
            ele_nums=ele_nums+1

        except:
            #print(element)
            
            continue
    if ele_nums != 15:
        print(job_info)
        #sys.exit()

    return job_listings


# List to store job listings
job_listings = []

# 获取总页数
total_pages_element = driver.find_element(By.XPATH, "//div[@class='dw_page']//span[contains(@class, 'td')]")
total_pages = int(total_pages_element.text.split('共')[-1].split('页')[0])

# 当前页数
current_page = 1

# 循环,遍历所有页面
while current_page <= total_pages:
    if current_page< total_pages:
        # 找到页面上所有特定的元素
        job_listings=get_information_from_elements(driver,job_listings)

        sleep(1)
        try:
            # 查找下一页按钮
            next_page_button = driver.find_element(By.XPATH, "//div[@class='dw_page']//a[text()='下一页']")
        except NoSuchElementException:
            sleep(3)
            next_page_button = driver.find_element(By.XPATH, "//div[@class='dw_page']//a[text()='下一页']")
            
        # 如果下一页按钮是不可点击的 span 元素，则说明当前页面已经是最后一页，退出循环
        if 'bk' in next_page_button.get_attribute('class'):
            break
        
        # 使用 JavaScript 来模拟点击下一页按钮
        driver.execute_script("arguments[0].click();", next_page_button)
        
        # 等待新页面加载完成
        sleep(2)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".e")))
        
        # 更新当前页数
        current_page += 1
    else:
        # 找到页面上所有特定的元素
        job_listings=get_information_from_elements(driver,job_listings)
        current_page += 1

# 将列表中的字典写入JSON文件
with open('34M_applied_job_listings_new.json', 'w', encoding='utf-8') as f:
    json.dump(job_listings, f, ensure_ascii=False, indent=4)


{'Job Title': '项目主管', 'Salary': '7千-1万', 'Company': '上海鑫冬环保科技有限公司', 'Location': '上海-闵行区', 'Application Time': '2024-04-08', 'Resume Status': '已到达'}


In [20]:
data24F=read_json_file(r'C:\Users\Xue\Documents\GitHub\gender-discrimination-hiring\Data_IT\new\29F_interested_job_listings.json')

In [ ]:
import sys

def get_information_from_elements(driver,job_listings):
    elements = driver.find_elements(By.CSS_SELECTOR, ".e")
    ele_nums=0
        # 遍历每个元素，提取并保存感兴趣的信息
    for element in elements:
        
        try:
            job_title = element.find_element(By.CSS_SELECTOR, "a.zhn").text
            salary = element.find_element(By.CSS_SELECTOR, "span.xz").text
            company = element.find_element(By.CSS_SELECTOR, "a.gs").text
            location = element.find_element(By.CSS_SELECTOR, "span.dq").text
            application_time = element.find_element(By.CSS_SELECTOR, ".li.l2 .rq span").text
            resume_status = element.find_element(By.CSS_SELECTOR, ".li.l4 .view span").text
            
            # 创建包含所有信息的字典
            job_info = {
                "Job Title": job_title,
                "Salary": salary,
                "Company": company,
                "Location": location,
                "Application Time": application_time,
                "Resume Status": resume_status
            }
            
            # 将字典添加到列表中
            job_listings.append(job_info)
            ele_nums=ele_nums+1

        except:
            #print(element)
            
            continue
    if ele_nums != 15:
        print(job_info)
        sys.exit()

    return job_listings


# List to store job listings
job_listings = []

# 获取总页数
total_pages_element = driver.find_element(By.XPATH, "//div[@class='dw_page']//span[contains(@class, 'td')]")
total_pages = int(total_pages_element.text.split('共')[-1].split('页')[0])

# 当前页数
current_page = 1
def catch_data(current_page,total_pages,job_listings,driver):
    # 循环,遍历所有页面
    while current_page <= total_pages:
        if current_page< total_pages:
            # 找到页面上所有特定的元素
            job_listings=get_information_from_elements(driver,job_listings)

            sleep(1)
            try:
                # 查找下一页按钮
                next_page_button = driver.find_element(By.XPATH, "//div[@class='dw_page']//a[text()='下一页']")
            except NoSuchElementException:
                sleep(3)
                next_page_button = driver.find_element(By.XPATH, "//div[@class='dw_page']//a[text()='下一页']")
                
            # 如果下一页按钮是不可点击的 span 元素，则说明当前页面已经是最后一页，退出循环
            if 'bk' in next_page_button.get_attribute('class'):
                break
            
            # 使用 JavaScript 来模拟点击下一页按钮
            driver.execute_script("arguments[0].click();", next_page_button)
            
            # 等待新页面加载完成
            sleep(2)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".e")))
            
            # 更新当前页数
            current_page += 1
        else:
            # 找到页面上所有特定的元素
            job_listings=get_information_from_elements(driver,job_listings)
            current_page += 1

    return job_listings

catch_data(current_page,total_pages,job_listings,driver2)


# 将列表中的字典写入JSON文件
with open('24F_applied_job_listings_new.json', 'w', encoding='utf-8') as f:
    json.dump(job_listings, f, ensure_ascii=False, indent=4)


In [12]:
job_listings

[{'Job Title': '人事行政主管',
  'Salary': '6-9千·13薪',
  'Company': '宁波明飞汽车零部件有限公司',
  'Location': '广州-南沙区',
  'Application Time': '2024-05-08',
  'Resume Status': '已查阅'},
 {'Job Title': '算法工程师（设备故障诊断）',
  'Salary': '2-2.5万',
  'Company': '泰尔重工股份有限公司',
  'Location': '上海-松江区',
  'Application Time': '2024-04-09',
  'Resume Status': '已到达'},
 {'Job Title': 'Python',
  'Salary': '1.2-1.8万',
  'Company': '广州碧软信息科技有限公司',
  'Location': '广州-天河区',
  'Application Time': '2024-04-09',
  'Resume Status': '已到达'},
 {'Job Title': '项目技术助理',
  'Salary': '7千-1.2万',
  'Company': '佩特库斯科技（北京）有限公司',
  'Location': '北京-朝阳区',
  'Application Time': '2024-04-09',
  'Resume Status': '已到达'},
 {'Job Title': '项目经理',
  'Salary': '1.5-2万',
  'Company': '网宿科技股份有限公司',
  'Location': '上海-嘉定区',
  'Application Time': '2024-04-09',
  'Resume Status': '已到达'},
 {'Job Title': '高级需求分析工程师',
  'Salary': '1.8-2.5万',
  'Company': '东莞市耀星网络技术有限公司',
  'Location': '北京-朝阳区',
  'Application Time': '2024-04-09',
  'Resume Status': '已到达'},
 {'Job 

In [8]:
data24F=read_json_file(r'C:\Users\Xue\Documents\GitHub\gender-discrimination-hiring\Data_IT\new\apply\24F_applied_job_listings.json')

In [5]:
driver2 = webdriver.Chrome(service=service,options=options)


In [15]:

def get_information_from_elements(driver,job_listings):
    elements = driver.find_elements(By.CSS_SELECTOR, ".txt")
    # 遍历找到的元素
    for element in elements:
        try:
            # 获取公司名称
            company_name = element.find_element(By.CSS_SELECTOR, ".li.l1 a").text
            
            # 获取HR兴趣度的宽度
            interest_width_style = element.find_element(By.CSS_SELECTOR, ".like span").get_attribute('style')
            interest_width = interest_width_style.split('width:')[1].replace('px;', '').strip()
            
            # 获取查看次数
            views_count = element.find_element(By.CSS_SELECTOR, ".li.l3 strong.c_orange").text
            
            # 获取最后一次查看的时间
            last_view_time = element.find_element(By.CSS_SELECTOR, ".li.l3 .f12 span.c_orange").text
            
            job_info={
                'company_name': company_name,
                'interest_width': interest_width,
                'views_count': views_count,
                'last_view_time': last_view_time
            }
            # 把提取的数据添加到列表中
            job_listings.append(job_info)

        except NoSuchElementException:
            # 如果没有找到某个元素，输出错误信息并跳过当前元素
            # print("Could not find one of the elements. Skipping.")
            continue
    return job_listings


job_listings = []


# 获取总页数
total_pages_element = driver.find_element(By.XPATH, "//div[@class='dw_page']//span[contains(@class, 'td')]")
total_pages = int(total_pages_element.text.split('共')[-1].split('页')[0])

# 当前页数
current_page = 1

# 循环,遍历所有页面
while current_page <= total_pages:
    if current_page< total_pages:
        # 找到页面上所有特定的元素
        job_listings=get_information_from_elements(driver,job_listings)

        sleep(2)
        try:
            # 查找下一页按钮
            next_page_button = driver.find_element(By.XPATH, "//div[@class='dw_page']//a[text()='下一页']")
        except NoSuchElementException:
            sleep(3)
            next_page_button = driver.find_element(By.XPATH, "//div[@class='dw_page']//a[text()='下一页']")
            
        # 如果下一页按钮是不可点击的 span 元素，则说明当前页面已经是最后一页，退出循环
        if 'bk' in next_page_button.get_attribute('class'):
            break
        
        # 使用 JavaScript 来模拟点击下一页按钮
        driver.execute_script("arguments[0].click();", next_page_button)
        
        # 等待新页面加载完成
        sleep(5)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".e")))
        
        # 更新当前页数
        current_page += 1
    else:
        # 找到页面上所有特定的元素
        job_listings=get_information_from_elements(driver,job_listings)
        current_page += 1


# 将列表中的字典写入JSON文件
with open('24M_interested_job_listings.json', 'w', encoding='utf-8') as f:
    json.dump(job_listings, f, ensure_ascii=False, indent=4)

In [18]:
import json
import os

# 已知的宽度到兴趣度的映射
width_to_interest_map = {
    8: 0.5,
    16: 1,
    30: 1.5,
    38: 2,
    52: 2.5,
    60: 3,
    74: 3.5,
    82: 4,
    96: 4.5,
    104: 5
}

# Python函数：根据interest_width值计算HR兴趣度（interest），并写入JSON数据
def calculate_interest(json_data):
    for entry in json_data:
        width_px = int(entry["interest_width"])
        # 检查宽度是否在映射表中
        if width_px in width_to_interest_map:
            entry["interest"] = width_to_interest_map[width_px]
        else:
            # 如果不在映射表中，则打印该项JSON数据
            print(f"公司：{entry['company_name']}，兴趣宽度：{entry['interest_width']}，宽度值不在映射表中")


# 指定文件夹路径
folder_path = r'C:\Users\Xue\Documents\GitHub\gender-discrimination-hiring\Data_IT\new'


# 初始化一个空列表来存储加载的数据
data_list = []

# 遍历文件夹中的所有文件
for filename in os.listdir(folder_path):
    # 检查文件是否为JSON文件
    if filename.endswith('.json'):
        # 构建完整的文件路径
        file_path = os.path.join(folder_path, filename)
        # 加载JSON数据
        with open(file_path, 'r', encoding='utf-8') as file:
            # 将加载的数据添加到列表中
            data_list.extend(json.load(file))

calculate_interest(data_list)

# 将处理后的数据保存到新的 JSON 文件中
with open(r'C:\Users\Xue\Documents\GitHub\gender-discrimination-hiring\Data_IT\new\new_interest.json', 'w', encoding='utf-8') as new_file:
    json.dump(data_list, new_file, ensure_ascii=False, indent=4)
